<h1 align="center"><font color="yellow">L1: NLP tasks with a simple interface  🖼️📝</font></h1>

<font color="yellow">Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro</font>

# <font color="red">Setup</font>

In [1]:
import os
import io
from IPython.display import Image, display, HTML
from PIL import Image
import base64 
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
Eddy_HUGGINGFACEHUB_API_TOKEN = os.environ['HUGGINGFACEHUB_API_TOKEN'] 


In [ ]:
# Helper function
import requests, json

#Summarization endpoint
def get_completion(inputs, parameters=None, ENDPOINT_URL="https://api-inference.huggingface.co/models/csebuetnlp/mT5_multilingual_XLSum"): 
    headers = {
      "Authorization": f"Bearer {Eddy_HUGGINGFACEHUB_API_TOKEN}",
      "Content-Type": "application/json"
    }
    data = { "inputs": inputs }
    if parameters is not None:
        data.update({"parameters": parameters})
    response = requests.request("POST",
                                ENDPOINT_URL, headers=headers,
                                data=json.dumps(data)
                               )
    return json.loads(response.content.decode("utf-8"))

# <font color="red">Building a text summarization app</font>

Usamos um modelo Multilingual: https://huggingface.co/csebuetnlp/mT5_multilingual_XLSum

## <font color="yellow">Que tal executá-lo localmente?</font>

O código seria muito semelhante se você o estivesse executando localmente em vez de uma `API`. O mesmo se aplica a todos os modelos do restante do curso, certifique-se de verificar a página de documentação do [Pipelines](https://huggingface.co/docs/transformers/main_classes/pipelines).

In [ ]:
from transformers import pipeline

get_completion = pipeline("summarization", model="csebuetnlp/mT5_multilingual_XLSum")

def summarize(input):
    output = get_completion(input)
    return output[0]['summary_text']


In [ ]:
text = ('''A torre tem 324 metros (1.063 pés) de altura, quase a mesma altura de um edifício de 81 andares, e é a estrutura mais alta de Paris. \
        Sua base é quadrada, medindo 125 metros (410 pés) de cada lado. \
        Durante sua construção, a Torre Eiffel ultrapassou o Monumento a Washington para se tornar a estrutura artificial mais alta do mundo, \
        título que manteve por 41 anos, até que o Edifício Chrysler na cidade de \
        Nova York foi concluído em 1930. Foi a primeira estrutura a alcançar uma altura de 300 metros. Devido à adição de uma antena de transmissão \
        no topo da torre em 1957, ela agora é 5,2 metros (17 pés) mais alta \
        que o Edifício Chrysler. Excluindo os transmissores, a Torre Eiffel é a segunda estrutura independente mais alta da França, depois do Viaduto Millau.''')


get_completion(text)

## <font color="yellow">Getting started with Gradio `gr.Interface`</font>

<font color="pink">Que tal executá-lo localmente?</font>

O código seria muito semelhante se você o estivesse executando localmente. Simplesmente remova todos os parâmetros do método launch

`demo.launch()`

In [ ]:
import gradio as gr

def summarize(input):
    output = get_completion(input)
    return output[0]['summary_text']
    
gr.close_all()

demo = gr.Interface(fn=summarize, inputs="text", outputs="text")
demo.launch(share=True, debug=True, server_port=7860)


# <font color="red">Building a Named Entity Recognition app</font>

<font color="yellow">A seguinte célula mostra de uma maneira diferente ao Tutorial. Eu fiz do meu jeito:</font>

In [ ]:
from transformers import pipeline

# Carregar o modelo NER (Reconhecimento de Entidades Nomeadas)
nlp_ner = pipeline("ner", model="dslim/bert-base-NER", tokenizer="dslim/bert-base-NER")

# Texto de exemplo
text = "My name is Andrew, I'm building DeepLearningAI and I live in California."

# Executar o NER no texto
result = nlp_ner(text)

# Exibir as entidades nomeadas reconhecidas
for entity in result:
    print(f"Entidade: {entity['entity']}, Texto: {entity['word']}, Score: {entity['score']}")



<font color="orange">Agora faço usando `Gradio`. Presta muita atenção neste exemplo com Gradio, já que aqui o TOKENS aparecem separados (as palavras aparecem separadas). </font>

In [ ]:
import gradio as gr
from transformers import pipeline

# Carregue o modelo NER usando a biblioteca transformers:
nlp_ner = pipeline("ner", model="dslim/bert-base-NER", tokenizer="dslim/bert-base-NER") # Usamos um modelo para o Inglês (não achei no HF para o Português)

def ner(input_text):
    # Use o modelo NER para processar o texto de entrada:
    result = nlp_ner(input_text)
    
    # Formate a saída no formato desejado (lista de entidades nomeadas):
    entities = []
    for ent in result:
        entities.append({
            "start": ent["start"],
            "end": ent["end"],
            "entity": ent["entity"],
            "score": ent["score"],
            "index": ent["index"],
        })
    return {"text": input_text, "entities": entities}


gr.close_all()
demo = gr.Interface(
    fn=ner,
    inputs=[gr.Textbox(label="Texto para encontrar entidades:", lines=2)],
    outputs=[gr.HighlightedText(label="Texto com entidades")],
    title="NER usando o modelo pré-treinado dslim/bert-base-NER",
    description="Encontre entidades usando o modelo `dslim/bert-base-NER`!",
    allow_flagging="never", #"manual", #"never",
    examples=["My name is Andrew and I live in California", "My name is Poli and work at HuggingFace"]
)
demo.launch(share=True, debug=True, server_port=7878) # Se você não adicionar a porta o GRADIO pega uma por default


## <font color="red">Adicionando uma função auxiliar para mesclar TOKENS</font>

In [14]:
import gradio as gr
from transformers import pipeline

def merge_tokens(tokens):
    merged_tokens = []
    for token in tokens:
        if merged_tokens and token['entity'].startswith('I-') and merged_tokens[-1]['entity'].endswith(token['entity'][2:]):
            # If current token continues the entity of the last one, merge them
            last_token = merged_tokens[-1]
            last_token['word'] += token['word'].replace('##', '')
            last_token['end'] = token['end']
            last_token['score'] = (last_token['score'] + token['score']) / 2
        else:
            # Otherwise, add the token to the list
            merged_tokens.append(token)

    return merged_tokens


# Carregue o modelo NER usando a biblioteca transformers:
nlp_ner = pipeline("ner", model="dslim/bert-base-NER", tokenizer="dslim/bert-base-NER") # Usamos um modelo para o Inglês (não achei no HF para o Português)

def ner(input_text):
    # Use o modelo NER para processar o texto de entrada:
    result = nlp_ner(input_text)
    merged_tokens = merge_tokens(result)
    # Formate a saída no formato desejado (lista de entidades nomeadas):
    # entities = []
    # for ent in result:
    #     entities.append({
    #         "start": ent["start"],
    #         "end": ent["end"],
    #         "entity": ent["entity"],
    #         "score": ent["score"],
    #         "index": ent["index"],
    #     })
    return {"text": input_text, "entities": merged_tokens}

gr.close_all()
demo = gr.Interface(
    fn=ner,
    inputs=[gr.Textbox(label="Texto para encontrar entidades:", lines=2)],
    outputs=[gr.HighlightedText(label="Texto com entidades")],
    title="NER usando o modelo pré-treinado dslim/bert-base-NER",
    description="Encontre entidades usando o modelo `dslim/bert-base-NER`!",
    allow_flagging="never", #"manual", #"never",
    examples=["My name is Andrew and I live in California", "My name is Poli and work at HuggingFace"]
)
demo.launch(share=True, server_port=4545) # Você escolhe a porta


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Closing server running on port: 7878
Closing server running on port: 4545
Closing server running on port: 7878
Closing server running on port: 7878
Closing server running on port: 7878
Closing server running on port: 4545
Running on local URL:  http://127.0.0.1:4545
Running on public URL: https://5dcafd32dfccd38b1c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
